# ДЗ 3. Коллаборативная фильтрация

## Теоретическая часть

### Задание 1.  
a. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи?  
b. Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  
c. В чем принципиальные отличия item-item рекомендаций от ALS?

a. User-User рекомендации - это метод, при котором определяется сходство между пользователями и в качестве рекомендаций пользователю выдается n самых часто покупаемых товаров, k наиболее похожими на него покупателями.  
Item-Item рекомендации - это метод, который ищет похожите товары на те, которые пользователь уже покупал и их рекомендует.  
Они похожи способом получения рекомендаций - из матрицы user - item мы создаем матрицу user-user (где на пересечении будут совпавшие товары) или item-item (на пересечении будут совпавшие пользователи). Различаются направленностью действий - в случае user-user мы находим похожих пользователей, а в item-item - товары.  

b. Да, функция есть. Использовать ее можно, только нужно транспорнировать матрицу.  

c. В item-item мы используем KNN, а в ALS матричное разложение. Помимо этого ALS формирет рекомендации в том числе для новых юзеров, item-item нет. ALS позволяет нам выиграть в скорости за счет уменьшения размерности, а также получить рекомендации даже если пользователь не имел взаимодействия с товаром, т.е. решить проблему холодного старта.


### Задание 2.  
Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров.


- популярные товары по времени суток
- количество просмотров страниц внутри категорий  
- факт добавления товара в корзину

### Задание 3.  
Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)

- Имеются ограничения по количеству обрабатываемых параметров, т.е. ALS не учитывает фичи юзеров и товаров  
- Нет возможности учитывать сезонность товара  

### Задание 4.  
Мы рассматривали bm_25_weight.  
Опишите, как он работает. Как сделать рекомендации только на основе bm_25? (Можно и нужно пользоваться любыми источниками, приложите на них ссылки). Какие еще способы перевзвешивания user-item матрицы Вы знаете / можете предложить (ещё 2-3 способа)?

_Как работает?_  

1. Вводится зависимость релевантности от вхождения или не вхождения слов в запросах более одного слова.  
2. Преимущество при поиске в запросах с более чем 2-ух слов, одно из которых менее употребительно (более узкоспециализированное) отдается документам, которые содержат это узкоспециализированное слово.  

_Способы перевзвешивания user-item матрицы:_  

- Помимо TF-IDF взвешивания user-item матрицы есть IDF-функция (обратная документная частота слова).  
- При экстремальных значениях коэффициента b в функции BM25 получаются функции ранжирования, известные под названиями BM11 (при b=1) и BM15 (при b=0).  
- BM25F — модификация BM25, в которой документ рассматривается как совокупность нескольких полей (таких как, например, заголовки, основной текст, ссылочный текст), длины которых независимо нормализуются, и каждому из которых может быть назначена своя степень значимости в итоговой функции ранжирования.  

_Источники:_  

https://habr.com/ru/post/162937/  
https://ru.wikipedia.org/wiki/Okapi_BM25

## Практическая часть


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

# Для работы с матрицами
from scipy.sparse import csr_matrix

import implicit

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# Положить  корень папку src    
from src.metrics import precision_at_k, recall_at_k

In [3]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [5]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [7]:
# Обучаем на 5000 товарах
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [8]:
# Заведем фиктивный item_id (если юзер не покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # кол-во покупок. Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Максимальное количество покупок
user_item_matrix.max().max()

2926.0

In [10]:
user_item_matrix.shape

(2499, 5001)

In [13]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [14]:
%%time

model = AlternatingLeastSquares(factors=100, # Нужно подбирать. Качество очень зависит.
                                # можно подбирать по кроссвалидации
                                regularization=0.001,
                                iterations=15, # кол-во итераций
                                calculate_training_loss=True, # отображать метрику loss в выводе
                                num_threads=4) # кол-во потоков распаралеливания

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=[itemid_to_id[999999]], 
                        # filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 1.81 s


In [15]:
# Посмотрим как выглядят рекомендации
recs

[(4346, 0.8028107630981962),
 (2148, 0.7744606925991713),
 (2927, 0.7141932326361788),
 (215, 0.6630146572161915),
 (2307, 0.6607898952469141)]

In [16]:
[id_to_itemid[rec[0]] for rec in recs]

[6534178, 981760, 1044078, 834484, 995242]

In [17]:
# Функция конвертации прогноза модели в нужные item id
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],
                                    #filter_items=None, 
                                    recalculate_user=True)]
    return res

In [18]:
result

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
...,...,...
2037,2496,[6534178]
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ..."
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972..."
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813..."


In [20]:
# Запускаем ALS и измеряем precision   
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

0.18227228207639307

In [21]:
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

0.18227228207639307

In [22]:
result.head(2)

,user_id,actual,als
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1033142, 995242, 832678, 979707, 9527290]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 5569327, 1133018, 910032, 5568378]"


### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


In [23]:
# Вес 1: nunique

# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

# заново создаём user-item матрицу, чтобы не применять 2 взвешивания подряд.
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='nunique', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
user_item_matrix = bm25_weight(user_item_matrix.T).T

In [26]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=8) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25_nunique'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25_nunique'], row['actual']), axis=1).mean()


Wall time: 24.4 s


0.21645445641527655

In [27]:
# Вес 2: mean

# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

# заново создаём user-item матрицу, чтобы не применять 2 взвешивания подряд.
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='mean', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
user_item_matrix = bm25_weight(user_item_matrix.T).T

In [29]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=8) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25_mean'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25_mean'], row['actual']), axis=1).mean()


Wall time: 20.1 s


0.21312438785504118

In [30]:
# Вес 3: quantity

# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

# заново создаём user-item матрицу, чтобы не применять 2 взвешивания подряд.
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='sum', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
user_item_matrix = bm25_weight(user_item_matrix.T).T

In [32]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=8) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25_quantity'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25_quantity'], row['actual']), axis=1).mean()


Wall time: 25.2 s


0.18452497551419894

In [33]:
result

,user_id,actual,als,als_bm25_nunique,als_bm25_mean,als_bm25_quantity
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1033142, 995242, 832678, 979707, 9527290]","[1082185, 1135694, 1033142, 8090541, 962568]","[1082185, 995242, 1027569, 1005186, 1056509]","[1082185, 6534178, 995242, 981760, 1029743]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 5569327, 1133018, 910032, 5568378]","[951590, 910032, 826249, 9337581, 5569230]","[1082185, 6534178, 1106523, 1029743, 951590]","[10285144, 6534178, 1029743, 1106523, 1133018]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1023720, 986912, 1024306, 866211, 981760]","[1082185, 878996, 1024306, 1127831, 1023720]","[1082185, 6534178, 1127831, 866211, 981760]","[1042697, 1082185, 965267, 1024306, 6534178]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[938700, 1082185, 1058997, 1126899, 1106523]","[1082185, 1127831, 879635, 873527, 1097544]","[1082185, 995242, 857390, 1077490, 1029743]","[1082185, 6534178, 995242, 1029743, 981760]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1068719, 938700, 844179, 913785, 1004906]","[1082185, 1068719, 883404, 826249, 840361]","[1082185, 995242, 1029743, 6534178, 1133018]","[1082185, 6534178, 995242, 1029743, 1106523]"
...,...,...,...,...,...,...
2037,2496,[6534178],"[914190, 995785, 1133018, 911812, 925626]","[1070702, 914190, 1000753, 1082185, 995785]","[6534178, 1082185, 916122, 995242, 981760]","[6534178, 1082185, 916122, 1133018, 981760]"
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[951590, 5569230, 854852, 5585510, 6534178]","[5569230, 981760, 1082185, 951590, 826249]","[5569230, 1082185, 951590, 1029743, 6534178]","[5569230, 1034176, 5569471, 6534178, 1082185]"
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[1053690, 862349, 6534178, 1082185, 1070820]","[951197, 824555, 6514160, 1053763, 1036432]","[6533889, 932006, 202291, 839747, 948086]","[982960, 6533889, 202291, 843464, 865456]"
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[6534178, 930917, 938700, 1004906, 965766]","[883404, 1098066, 826249, 951590, 930917]","[6534178, 6533889, 1082185, 1029743, 1106523]","[6534178, 6533889, 1082185, 5569230, 883404]"


**Вывод.** Без дополнительной предобработки данных, использование разных матриц весов не даёт прироста по результату!

### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.GridSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

In [34]:
%%time

for factors in tqdm(np.linspace(30,170,8)):
    for reg in np.geomspace(0.0001,0.01,8):

        # Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
        data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

        # заново создаём user-item матрицу, чтобы не применять 2 взвешивания подряд.
        user_item_matrix = pd.pivot_table(data_train, 
                                          index='user_id', columns='item_id', 
                                          values='quantity', # Можно пробовать другие варианты
                                          aggfunc='count', 
                                          fill_value=0
                                         )

        user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

        # переведем в формат saprse matrix
        sparse_user_item = csr_matrix(user_item_matrix).tocsr()

        user_item_matrix.head(3)

        user_item_matrix = bm25_weight(user_item_matrix.T).T

        

        model = AlternatingLeastSquares(factors=int(factors), 
                                        regularization=reg,
                                        iterations=15, 
                                        calculate_training_loss=True, 
                                        num_threads=8) # K - кол-во билжайших соседей

        model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
                  show_progress=False)

        result[f'als_bm25_factors={factors}_reg={reg}'] = \
                        result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

        result.apply(lambda row: precision_at_k(row[f'als_bm25_factors={factors}_reg={reg}'], \
                                                            row['actual']), axis=1).mean()

  0%|          | 0/8 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
 38%|███▊      | 3/8 [05:45<09:13, 110.61s/it]C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

Wall time: 25min 51s


In [35]:
result.head()

,user_id,actual,als,als_bm25_nunique,als_bm25_mean,als_bm25_quantity,als_bm25_factors=30.0_reg=0.0001,als_bm25_factors=30.0_reg=0.00019306977288832496,als_bm25_factors=30.0_reg=0.0003727593720314938,als_bm25_factors=30.0_reg=0.0007196856730011522,...,als_bm25_factors=150.0_reg=0.005179474679231213,als_bm25_factors=150.0_reg=0.01,als_bm25_factors=170.0_reg=0.0001,als_bm25_factors=170.0_reg=0.00019306977288832496,als_bm25_factors=170.0_reg=0.0003727593720314938,als_bm25_factors=170.0_reg=0.0007196856730011522,als_bm25_factors=170.0_reg=0.0013894954943731374,als_bm25_factors=170.0_reg=0.0026826957952797246,als_bm25_factors=170.0_reg=0.005179474679231213,als_bm25_factors=170.0_reg=0.01
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1033142, 995242, 832678, 979707, 9527290]","[1082185, 1135694, 1033142, 8090541, 962568]","[1082185, 995242, 1027569, 1005186, 1056509]","[1082185, 6534178, 995242, 981760, 1029743]","[1082185, 1004390, 995242, 1051211, 1100972]","[1082185, 1004390, 995242, 1051211, 878996]","[1082185, 1004390, 1051211, 995242, 1082212]","[1082185, 995242, 1082212, 999104, 862349]",...,"[965766, 1082185, 995242, 856942, 934369]","[1082185, 995242, 965766, 1051211, 9527290]","[995242, 1082185, 867188, 934369, 856942]","[965766, 1082185, 995242, 856942, 5569374]","[995242, 1082185, 965766, 934369, 1033142]","[1082185, 995242, 1051211, 1033142, 965766]","[1082185, 995242, 965766, 9527290, 1033142]","[995242, 965766, 1082185, 1033142, 856942]","[995242, 1082185, 965766, 1051211, 1033142]","[1082185, 995242, 965766, 904360, 934369]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 5569327, 1133018, 910032, 5568378]","[951590, 910032, 826249, 9337581, 5569230]","[1082185, 6534178, 1106523, 1029743, 951590]","[10285144, 6534178, 1029743, 1106523, 1133018]","[951590, 8090521, 1053690, 844165, 1092026]","[951590, 8090521, 962229, 883404, 5569230]","[951590, 8090521, 5569230, 962229, 883404]","[951590, 1053690, 5569230, 962229, 826249]",...,"[1133018, 1053690, 1106523, 1092026, 951590]","[1133018, 1106523, 1053690, 1092026, 951590]","[1133018, 1106523, 1053690, 883404, 1092026]","[1133018, 1053690, 883404, 1092026, 1022003]","[1133018, 1106523, 1053690, 1098066, 1092026]","[1133018, 1053690, 1092026, 1106523, 951590]","[1133018, 1106523, 1092026, 951590, 1053690]","[1133018, 1106523, 1092026, 1053690, 910032]","[1133018, 1092026, 1106523, 951590, 1053690]","[1133018, 1053690, 1092026, 951590, 883404]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1023720, 986912, 1024306, 866211, 981760]","[1082185, 878996, 1024306, 1127831, 1023720]","[1082185, 6534178, 1127831, 866211, 981760]","[1042697, 1082185, 965267, 1024306, 6534178]","[1082185, 1024306, 857006, 1007195, 878996]","[1082185, 857006, 1024306, 878996, 1023720]","[1082185, 878996, 857006, 1024306, 1023720]","[1082185, 857006, 1024306, 878996, 834484]",...,"[1082185, 878996, 1024306, 919681, 1007195]","[1082185, 866211, 1024306, 1023720, 1051516]","[1082185, 878996, 1024306, 1098248, 1023720]","[878996, 1082185, 1007195, 866211, 1051516]","[1082185, 1023720, 1007195, 878996, 1024306]","[1082185, 1024306, 866871, 1007195, 1098248]","[866211, 1024306, 878996, 1082185, 1007195]","[1023720, 904360, 1082185, 1007195, 878996]","[1082185, 878996, 1007195, 1023720, 866871]","[1023720, 1082185, 1024306, 878996, 1007195]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[938700, 1082185, 1058997, 1126899, 1106523]","[1082185, 1127831, 879635, 873527, 1097544]","[1082185, 995242, 857390, 1077490, 1029743]","[1082185, 6534178, 995242, 1029743, 981760]","[1082185, 849843, 6944571, 1130111, 1058997]","[1082185, 849843, 6944571, 1058997, 5981267]","[1082185, 849843, 6944571, 1058997, 893018]","[1082185, 849843, 6944571, 1058997, 5981267]",...,"[1082185, 6944571, 1126899, 1122358, 849843]","[1082185, 6944571, 1122358, 826249, 849843]","[1082185, 6944571, 1126899, 893018, 826249]","[1082185, 828867, 6944571, 1122358, 1058997]","[1082185, 828867,

In [36]:
df_res = pd.DataFrame(columns=['parameters', 'presision_at_k'])
i = 0
for column in result.columns[2:]:
    a = round(result.apply(lambda row: precision_at_k(row[column], row['actual']), axis=1).mean(), 3)
    df_res.loc[i] = [column, a]
    i += 1

In [37]:
df_res.sort_values(by='presision_at_k', ascending=False).head(10)

,parameters,presision_at_k
62,als_bm25_factors=170.0_reg=0.0003727593720314938,0.235
52,als_bm25_factors=150.0_reg=0.0001,0.234
66,als_bm25_factors=170.0_reg=0.005179474679231213,0.234
57,als_bm25_factors=150.0_reg=0.0026826957952797246,0.233
56,als_bm25_factors=150.0_reg=0.0013894954943731374,0.233
59,als_bm25_factors=150.0_reg=0.01,0.232
63,als_bm25_factors=170.0_reg=0.0007196856730011522,0.231
50,als_bm25_factors=130.0_reg=0.005179474679231213,0.231
51,als_bm25_factors=130.0_reg=0.01,0.231
55,als_bm25_factors=150.0_reg=0.0007196856730011522,0.230


**Вывод:**  
Оптимальные параметры для модели:  
factors = 170, lambda = 0.01